In [16]:
## Importing libraries
from datetime import date
from bs4 import BeautifulSoup
import requests
import pandas as pd
import xlrd #This library is necessary because of the the xls files.

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0"}

URL = "https://www.inide.gob.ni/Home/canasta"
page = requests.get(URL, headers = headers,verify=False)
Soup1 = BeautifulSoup(page.content, "html.parser")
Soup2 = BeautifulSoup(Soup1.prettify(),"html.parser")

In [18]:
row = []
for fila in Soup2.find_all("a", href = True):
    row.append(fila["href"])

In [19]:
## Getting the URLs data. Each URL redirects to xls or xlsx files
cleaned_URL = []
url_main = "https://www.inide.gob.ni"

for i in row:
    if i.startswith("/docs"):
        cleaned_URL.append(url_main + i)
    else:
        continue

In [46]:
## This function gets the month from the files. They repeat the same pattern so this approach can be useful

def Month_Date(x):
    Months = {
    "ene":"-01-01",
    "feb":"-02-01",
    "mar":"-03-01",
    "abr":"-04-01",
    "may":"-05-01",
    "jun":"-06-01",
    "jul":"-07-01",
    "ago":"-08-01",
    "sep":"-09-01",
    "oct":"-10-01",
    "nov":"-11-01",
    "dic":"-12-01"
    }

    Month = None
    Text = x.split("/")[-1]

    for i in Months.keys():
        if i in Text.lower():
            Month = Months[i]
            break
        else:
            continue
    return Month


In [53]:
## Describing the column names for the dataframe
columns_df = ["Descripción de productos", "Unidad de Medida", "Cantidad Mensual de Consenso", "Precios", "Córdobas", "Fecha"]

## THis function extract, transforms and load the data into a dataframe
def data_ETL (df,df_date = None):   
    df = df.iloc[4:]
    df = df.drop("Unnamed: 0", axis = 1).dropna().reset_index(drop=True)
    df["Fecha"] = df_date
    df.columns = columns_df
    return df

In [56]:
## Main Dataframe where data is loaded
df = pd.DataFrame(columns = columns_df)

## Getting the data, passing it through the functions and loading them to the dataframe
for i in cleaned_URL:
    try: 
        s = requests.get(i).content
        df_date = i.split("/")[-2][2:] + Month_Date(i)
        df_new_data = pd.read_excel(s)
        df_new_data = data_ETL(df_new_data, df_date)
        df = pd.concat([df,df_new_data],ignore_index=True)
    except:
        print("Failed: " + i )
        break



In [60]:
## Exporting to csv file
df.to_csv("CB Data_ISO.csv", encoding= "ISO 8859-1")